In [1]:
# Monte Carlo Simulation Build
# non-normal multivariant distributions

In [1]:
import pandas as pd
from pandas_datareader import data as pdr

import numpy as np

import matplotlib.pyplot as plt
import datetime as dt

from scipy import stats

import yfinance as yf

In [2]:
# specifiy our date range for data
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days = 1825)
start, end = start_date, end_date
start, end                                       # verify our date range

(datetime.datetime(2017, 10, 14, 13, 47, 52, 675364),
 datetime.datetime(2022, 10, 13, 13, 47, 52, 675364))

In [3]:
asset_list = ['AAPL', 'IBM', 'TSLA', 'GOOGL']
stocks = ['^GSPC'] + asset_list                              # adding S&P 500 index (^GSPC)for use in beta weighting
stocks

['^GSPC', 'AAPL', 'IBM', 'TSLA', 'GOOGL']

## yahoo access via panda datareader-
###    pdr.DataReader(stocks, 'yahoo', start, end)
### or
###    pdr.get_data_yahoo(stocks, start, end)

In [11]:
# get the data
def get_the_data(stocks, start, end):
    stock_data = pdr.get_data_yahoo(stocks, start, end)
    stock_data = stock_data['Close']
    returns = stock_data.pct_change()
    mean_returns = returns.mean()
    log_returns = np.log(stock_data.Close / stock_data.Close.shift(1)).dropna()
    stock_covariance = returns.cov()
    return stock_data, mean_returns, log_returns, stock_covariance

In [12]:
# Confirm log returns for beta
# display(log_returns.shape)
# display(log_returns.head())
stock_data, mean_returns, log_returns, stock_covariance = get_the_data(stocks, start, end)
display(stock_data.head())

AttributeError: 'DataFrame' object has no attribute 'Close'

In [ ]:
# 'Market'as a whole is S&P 500 index = GSPC
# DIRECT BETA CALCULATION =>      covariance(GSPC, stock) /  variance(GSPC)  ; note variance of S&P500 = covariance(S&P500, S&P500)   

In [ ]:
# Beta
def direct_beta(log_returns):
    np_array = dataframe.values
    # S&P 500 index (^GSPC) is in first column, 0
    m = np_array[:,0]
    beta = []
    for index, col in enumerate(dataframe):
        if index > 0:
            # asset returns indexed by index
            s = np_array[:,index]
            # calculating covariance between assets and S&P 500 index (GSPC)
            covariance = np.cov(s, m)
            beta.append(covariance[0,1] / covariance[1,1])

    return pd.Series(beta, dataframe.columns[1:], name = 'Beta')

direct_beta(log_returns)

In [ ]:
# get the data
def get_the_data(stocks, start, end):
    stock_data = pdr.get_data_yahoo(stocks, start, end)
    stock_data = stock_data['Close']
    returns = stock_data.pct_change()
    mean_returns = returns.mean()
    stock_covariance = returns.cov()
    return mean_returns, stock_covariance

In [ ]:
asset_list = ['AAPL', 'IBM', 'TSLA', 'GOOGL']
stocks = asset_list
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days = 365)

In [ ]:
mean_returns,  stock_covariance = get_the_data(stocks, start_date, end_date)
print(mean_returns)

In [ ]:
# random weighting
weight = np.random.random(len(mean_returns))   # derives number between 0 and 1
weight /= np.sum(weight)
print(weight)

In [ ]:
# Monte Carlo Simulattion Generator

In [ ]:
# number of simulations
mc_sims = 100
# timeframe in days
T = 365

In [ ]:
mean_m = np.full(shape=(T, len(weight)), fill_value=mean_returns)   #  shape is T(timeframe days) and number of weight
mean_m = mean_m.T     # transpose

In [ ]:
# array/matrix to store the information
portfolio_simulations = np.full(shape=(T, mc_sims), fill_value=0.0)  # shape is factor of time and number of sims; fill = 0.0 allows for floats

In [ ]:
# simulate initial value
starting_portfolio_value = 10000

In [ ]:
#.     check cholesky decomposition triangle matrix
for m in range(0, mc_sims):
    # Monte Carlo loops
    Z = np.random.normal(size = (T, len(weight)))   # T x number assets uncorrelated random variables from norm distribution
    L = np.linalg.cholesky(stock_covariance)              # lower triangle for cholesky decomp
    daily_returns = mean_m + np.inner(L, Z)
    portfolio_simulations[:,m] = np.cumprod(np.inner(weight, daily_returns.T)+1) * starting_portfolio_value

In [ ]:
# visualize
plt.plot(portfolio_simulations)
plt.ylabel('Value of Portfolio ($)')
plt.xlabel('Days')
plt.title('Monte Carlo Simulation Portfolio Performance')
plt.show

In [ ]:
# Value at risk and conditional value at risk

In [ ]:
def mc_value_at_risk(returns, alpha = 5):    #alpha default fifth percentile
    # input: panda series of returns
    # output: percentile on return distribution to a given confidence level alpha
    if isinstance(returns, pd.Series):
        return np.percentile(returns, alpha)
    else:
        raise TypeError('expected a panda data series')


In [ ]:
# conditional value at risk - expected shortfall once passed beyond the percentile of interest
#                             to a certain confidence interval what is our expected shortfall below that percentile
def mc_conditional_value_at_risk(returns, alpha = 5):    #alpha default fifth percentile
    # input: panda series of returns
    # output: mc_conditional_value_risk or expected shortfall to a given confidence level alpha
    if isinstance(returns, pd.Series):
        below_value_at_risk = returns <= mc_value_at_risk(returns, alpha = alpha)
        return returns[below_value_at_risk].mean()
    else:
        raise TypeError('expected a panda data series')

In [ ]:
#
portfolio_results = pd.Series(portfolio_simulations[-1,:])

In [ ]:
value_at_risk = starting_portfolio_value - mc_value_at_risk(portfolio_results, alpha = 5)
conditional_value_at_risk = starting_portfolio_value - mc_conditional_value_at_risk(portfolio_results, alpha = 5)

print('Value at Risk ${}'.format(round(value_at_risk,2)))
print('Conditional Value at Risk ${}'.format(round(conditional_value_at_risk,2)))